*This code calculates the distance of each snail occurrence point to rural areas (defined by the UN)*

In [ ]:
# this code will create rasters that is distance to urban edges in Brazil
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

**SET UP DATA**

In [ ]:
## set up the variables to include in the function
#Brazil feature
region = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level0').filter(ee.Filter.equals('ADM0_NAME', 'Brazil'));

# Read in snail points
all_sdm_points = ee.FeatureCollection('users/cglidden/all_points_schisto_sdm')

def buffer_points(radius, bounds):
    def buffer_feature(pt):
        pt = ee.Feature(pt)
        return ee.Algorithms.If(bounds, pt.buffer(radius).bounds(), pt.buffer(radius))
    return buffer_feature

# Asset of regions for which you want to calculate statistics - radius of 1km2
bufferedPoints = all_sdm_points.map(buffer_points(500, True))

# Load the image collection
worldPop = (ee.ImageCollection("WorldPop/GP/100m/pop")
            .filterMetadata('country', 'equals', 'BRA')
            .toBands()
            .unmask(ee.Image().toByte().paint(region.geometry(), 1).multiply(0))
            .clip(region))

# Convert WP to 1km
projection = worldPop.projection()
projectionAt1k = projection.atScale(1000)

pop1k = (worldPop.reduceResolution(
            reducer=ee.Reducer.sum().unweighted(),
            maxPixels=1024)
        .reproject(crs=projectionAt1k))



**CONVERT WORLD POP DATA TO RURAL (0,1)**

In [ ]:
def mapRural(year):

    yearString = ee.Number(year).toInt().format()
    band_name = ee.String("BRA_").cat(yearString).cat("_population")

    ruralMax = 300

    ruralMask = (pop1k.select([band_name]).lt(ruralMax)
    .rename(ee.String("rural_").cat(ee.Number(year).toInt().format())));

    return ee.Image(ruralMask)

def remove_index(band_name):
    return ee.String(band_name).slice(-10)


**RUN FUNCTION**

In [ ]:
years = ee.List.sequence(2000, 2020);

rural_binary_collection = ee.ImageCollection(years.map(mapRural));
rural_binary = rural_binary_collection.toBands()

# Get the band names
band_names = rural_binary.bandNames()

# Apply the function to each band name
new_band_names = band_names.map(remove_index)

# Rename the bands
renamed_rural = rural_binary.select(band_names, new_band_names)

**FINAL CUMULATIVE COST MAPPING GeoTiffs**

In [ ]:
##### FINAL COST MAPPING
def mapCost (year):

    yearString = ee.Number(year).toInt().format();
    band_name = ee.String("rural_").cat(yearString);

    final_rural_binary = renamed_rural.select([band_name]);
    final_rural_binary_v2 = final_rural_binary.gt(0) # maybe unneccesary

    # set masked values to 50km
    overRural = ee.Image().toByte().paint(region.geometry(), 50000)

    # create even surface
    landscape = ee.Image().toByte().paint(region.geometry(), 1)
    cumulativeCostRural0 = (landscape.cumulativeCost(
                            source=final_rural_binary_v2,
                            maxDistance= 50 * 1000,
                            geodeticDistance = False)
                         .rename(ee.String("rural_").cat(ee.Number(year).toInt().format()))
                         .unmask(overRural))

    # convert anything greater than 50km to 50km
    condition1 = cumulativeCostRural0.lte(50000); #50,000 meters = 50 km
    cumulativeCostRural = cumulativeCostRural0.updateMask(condition1).unmask(50000).clip(region);

    return ee.Image(cumulativeCostRural)


**EXPORT COST MAPS AS IMAGES - TRY TO RUN IN ONE OUTPUT**

In [ ]:
years = ee.List.sequence(2000, 2020) # Modify this list as needed

cc_images = ee.ImageCollection(years.map(mapCost))
cc_image = cc_images.toBands()

# updated bands
cc_names = cc_image.bandNames()
new_cc_names = cc_names.map(remove_index)
renamed_cc = cc_image.select(cc_names, new_cc_names)

# asset ID
cc_id = 'projects/gbsc-gcp-lab-emordeca/assets/urban_mapping/ruralCostMap'

# Save the collection
task = ee.batch.Export.image.toAsset(
    image= renamed_cc,
    description ='ruralCostMap',
    assetId = cc_id,
    region=region.geometry(),
    scale=1000,
    crs='EPSG:4326'
)
task.start()

**FINAL FEATURE EXPORT**

In [ ]:
# read in images from mapRural
cost_maps = ee.Image('projects/gbsc-gcp-lab-emordeca/assets/urban_mapping/ruralCostMap')

reducer2 = ee.Reducer.mean()

finalFeature = cost_maps.reduceRegions(
      collection = bufferedPoints,
                      reducer = reducer2,
                      scale = 1000,
                      crs = 'EPSG:4326',
                      tileScale = 16);

# Export the image sample feature collection to Drive as a CSV file.
task = ee.batch.Export.table.toDrive(
    collection=finalFeature,
    description='schisto_ruralCC_oct102023',
    folder='final_schisto_data',
    fileFormat='CSV',
)
task.start()


**NOW READ IN CSV & CLEAN**

In [ ]:
# acess the file & set up file
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# read in data
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/GEEexports brazil_schisto_snails/final_schisto_data/schisto_ruralCC_oct102023.csv')
# print(df.head())
print(df.columns)

In [ ]:
# subset data to make it easier to switch from wide to long
desired_indices = list(range(3, 25)) + [27]
df_sub = df.iloc[:, desired_indices] # might have to update this
print(df_sub.columns)


In [ ]:
#### now convert wide to long dataset, merge data, and export (print len to make sure merged okay)
df_long = pd.melt(df_sub, id_vars=['row_code', 'year'], var_name='year2', value_name='dist_rural')
df_long = df_long.dropna()
df_long['year2'] = df_long['year2'].str.extract(r'(\d+)')
df_long['year2'] = pd.to_numeric(df_long['year2'])
df_long = df_long[(df_long['year'] == df_long['year2'])]
df_long = df_long.drop(columns=['year2'])
print(df_long.head())
print(len(df_long)) # no. of rows

# sanity check - check years
unique_values = df_long['year'].unique()
print("Unique Years :", unique_values)

In [ ]:
df_long.to_csv('/content/drive/MyDrive/GEEexports brazil_schisto_snails/final_schisto_data/cleaned_schisto_ruralCC_oct102023.csv', index=False)
